# Recuperação da Informação e Busca na Web - 2018.1

### Atividade: Lab 02 - Parte 2 - PageRank
### Aluno: Johanny de Lucena Santos

## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Nesta atividade você vai exercitar o algoritmo PageRank. De forma específica, você vai fazer o seguinte:

<ul>
    <ul>
        <li>Assistir o vídeo e (re)implementar o PageRank como descrito neste vídeo (https://www.youtube.com/watch?v=zv4OVNWfVt4).

<ul>
    <li>É importante que você realmente digite o código durante o vídeo (pausando quando necessário), isso é mais efetivo para o aprendizado do que somente copiar e colar do github por exemplo. Tente entender o código, ele é bem próximo ao que foi explicado em sala de aula.</li>

<li>Essa é somente uma das muitas implementações do PageRank disponíveis na Web. Você pode escolher outra se assim desejar.</li></ul>
<p>   
<li>Aplique sua implementação nos dados disponíveis em http://snap.stanford.edu/data/soc-sign-bitcoinotc.html. Esses dados representam links entre investidores de bitcoin. Cada investidor pode receber uma nota de acordo com outras pessoas que fizeram transações com ele(a). **Vamos considerar somente as pessoas que receberam notas iguais ou maior a oito. Ou seja, somente as arestas que tenham peso maior ou igual à oito**.
    <p>
<li>Uma vez calculado o PageRank, gere visualizações dos resultados usando a ferramenta Gephi (https://gephi.org/).
   <br> 
    <br>
Feitos os passos acima, responda as seguintes perguntas:
<p>
<li>Quantas iterações o PageRank precisou rodar até atingir convergência?</li>
<li>Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?</li>
<li>Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?</li>

### Importações

In [2]:
import numpy as np
import pandas as pd
from fractions import Fraction

@inproceedings{kumar2016edge,
  title={Edge weight prediction in weighted signed networks},
  author={Kumar, Srijan and Spezzano, Francesca and Subrahmanian, VS and Faloutsos, Christos},
  booktitle={Data Mining (ICDM), 2016 IEEE 16th International Conference on},
  pages={221--230},
  year={2016},
  organization={IEEE}
}

## Definição do dataset a ser utilizado

In [10]:
dataset = pd.read_csv('../data/soc-sign-bitcoinotc.csv')
print("Source ID - Target ID - Rating - Time")
dataset.head()

Source ID - Target ID - Rating - Time


,6,2,4,1289241911.72836
0,6,5,2,1.289242e+09
1,1,15,1,1.289243e+09
2,4,3,7,1.289245e+09
3,13,16,8,1.289254e+09
4,13,10,8,1.289254e+09


### Filtrar os dados
Vamos considerar somente as pessoas que receberam notas iguais ou maior a oito. Ou seja, somente as arestas que tenham peso maior ou igual à oito.
<p>
Ou seja, filtrar os dados para os elementos que possuem a coluna Rating (3ª) maior ou igual a 8

### Algoritmo PageRank do vídeo

In [11]:
'''
    Original transition matrix a, with A=0, B=1, ...
    Matriz de Transição estocástica
    Começa com 1 onde tem links de saida, depois divide pelo nº de saidas
'''

h = np.matrix([
    [ 0,    0,      0,       0,      0,     0],
    [1.0,   0,   1.0/3.0, 1.0/3.0, 1.0/4.0, 0],
    [ 0, 1.0/3.0,   0,    1.0/3.0, 1.0/4.0, 0],
    [ 0, 1.0/3.0, 1.0/3.0,   0,    1.0/4.0, 0],
    [ 0, 1.0/3.0, 1.0/3.0, 1.0/3.0,   0,    0],
    [ 0,    0,       0,       0,   1.0/4.0, 0]])

print ('H:', '\n', h)

H: 
 [[ 0.          0.          0.          0.          0.          0.        ]
 [ 1.          0.          0.33333333  0.33333333  0.25        0.        ]
 [ 0.          0.33333333  0.          0.33333333  0.25        0.        ]
 [ 0.          0.33333333  0.33333333  0.          0.25        0.        ]
 [ 0.          0.33333333  0.33333333  0.33333333  0.          0.        ]
 [ 0.          0.          0.          0.          0.25        0.        ]]


In [12]:
'''
    Google matrix with domping factor=0.15
'''
factor = float(1)/float(6)
d = 0.15

b = factor * np.matrix([
    [1,1,1,1,1,1],
    [1,1,1,1,1,1],
    [1,1,1,1,1,1],
    [1,1,1,1,1,1],
    [1,1,1,1,1,1],
    [1,1,1,1,1,1]])

# M = (1-d)*H + d*B, d = 0.15
m = (1-d) * h + d * b
print ('M:','\n', m)

M: 
 [[ 0.025       0.025       0.025       0.025       0.025       0.025     ]
 [ 0.875       0.025       0.30833333  0.30833333  0.2375      0.025     ]
 [ 0.025       0.30833333  0.025       0.30833333  0.2375      0.025     ]
 [ 0.025       0.30833333  0.30833333  0.025       0.2375      0.025     ]
 [ 0.025       0.30833333  0.30833333  0.30833333  0.025       0.025     ]
 [ 0.025       0.025       0.025       0.025       0.2375      0.025     ]]


In [13]:
'''
    Original normalized vector v
'''
factor = float(1)/float(6)

v = factor * np.matrix([
    [1],
    [1],
    [1],
    [1],
    [1],
    [1]])

print('v:', '\n', v)

v: 
 [[ 0.16666667]
 [ 0.16666667]
 [ 0.16666667]
 [ 0.16666667]
 [ 0.16666667]
 [ 0.16666667]]


In [14]:
'''
    Define the PageRank function. It does m*v and recursive calls itself until it converges.
    The converge difference (sum of difference of every elements) is set to 0.001
'''

count = 0

def pageRank(v):
    global count
    
    if sum(abs(m*v-v)) > 0.001:
        count += 1
        print('count:', count)
        print (m*v)
        print('sum(abs(m*v-v)):', sum(abs(m*v-v)))
        return pageRank(m*v)
    
    else:
        count += 1
        print('count:', count)
        print(m*v)
        print('sum(abs(m*v-v)):', sum(abs(m*v-v)))
        return m*v
        

In [15]:
# Chama a função pageRank
result = pageRank(v)

count: 1
[[ 0.025     ]
 [ 0.29652778]
 [ 0.15486111]
 [ 0.15486111]
 [ 0.16666667]
 [ 0.06041667]]
sum(abs(m*v-v)): [[ 0.40138889]]
count: 2
[[ 0.02145833]
 [ 0.16587963]
 [ 0.18476852]
 [ 0.18476852]
 [ 0.19322917]
 [ 0.056875  ]]
sum(abs(m*v-v)): [[ 0.2241088]]
count: 3
[[ 0.02017448]
 [ 0.18417742]
 [ 0.16058599]
 [ 0.16058599]
 [ 0.17187587]
 [ 0.06123568]]
sum(abs(m*v-v)): [[ 0.09366069]]
count: 4
[[ 0.01896589]
 [ 0.16363654]
 [ 0.15317247]
 [ 0.15317247]
 [ 0.16214821]
 [ 0.05548951]]
sum(abs(m*v-v)): [[ 0.05205033]]
count: 5
[[ 0.01766463]
 [ 0.15503986]
 [ 0.14188368]
 [ 0.14188368]
 [ 0.15082605]
 [ 0.05212112]]
sum(abs(m*v-v)): [[ 0.04716608]]
count: 6
[[ 0.01648548]
 [ 0.14395169]
 [ 0.13266435]
 [ 0.13266435]
 [ 0.14081419]
 [ 0.04853601]]
sum(abs(m*v-v)): [[ 0.04430295]]
count: 7
[[ 0.0153779 ]
 [ 0.13449003]
 [ 0.12367546]
 [ 0.12367546]
 [ 0.13134068]
 [ 0.04530092]]
sum(abs(m*v-v)): [[ 0.04125561]]
count: 8
[[ 0.01434651]
 [ 0.12541038]
 [ 0.11540329]
 [ 0.11540329]
 

In [16]:
# Imprime o resultado ordenado

print ('Sorted Result:', '\n', sorted(result, reverse=True))
print ('B', 'E', 'C', 'D', 'F', 'A')

Sorted Result: 
 [matrix([[ 0.00316083]]), matrix([[ 0.003088]]), matrix([[ 0.00290815]]), matrix([[ 0.00290815]]), matrix([[ 0.00106494]]), matrix([[ 0.00036155]])]
B E C D F A


In [19]:
print(result)

[[ 0.00036155]
 [ 0.00316083]
 [ 0.00290815]
 [ 0.00290815]
 [ 0.003088  ]
 [ 0.00106494]]
